In [ ]:
cc.VerificationHandler.close_browser()

In [133]:
VerificationHandler.close_browser()

In [1]:
import numpy as np
import pandas as pd
from bs4.element import NavigableString

In [2]:
% run contactsScraper.py

Done
Done
KEYS
Done
Done
RECORDS COLLECTED
DATAFRAMES READY


In [3]:
% run contactChecker.py

Local Contact Checker Ready


In [4]:
ContactSheetOutput.set_output(contactKeys)
VerificationHandler.set_orgRecords(dm.OrgSession(orgRecords))
VerificationHandler.set_contactRecords(cr)
print('Local Contact Checker Ready')

RECORD OUTPUT READY
Local Contact Checker Ready


In [5]:
class ContactScraperVerifier(MotherSetVerifier):
    def __init__(self, org):
        MotherSetVerifier.__init__(self, org)
        self.vPointers = self.verifiedPointers
        
        # Setup the NewContactSheetOutput Class for new records- Send it one of the contact records for the organization
        NewContactSheetOutput.set_Org_Fill_In_Fields(self.records.iloc[[0]])
        
        # Get Grand Mother elements
        self.gmElements, self.grandMotherMatrix = ContactScraperVerifier.getGrandMotherElements(self.vPointers)
        
        self.noGm = len(self.gmElements)
        
        if self.noGm == 1: ## Single Grandmother Case
            self.gm = self.gmElements[0]
            self.distinct_gm = ContactScraperVerifier.distinct_gm(self.gm, self.vPointers)
            
            #if ContactScraperVerifier.distinct_gm(self.gm, self.pointers): ## Grandmother is distinct
            ### Extender Model Selection
            #    self.extenders = [Extender(self.gm, vp) for vp in self.pointers]
            #
            #else:  ## GrandMother is not distinct
            #    ## Extender Model Selection
            #    self.extenders = [RocketOnlyExtender(self.gm, vp) for vp in self.pointers]
            
        else:
            ## Extender Model Selection for Multiple GrandMothers
            self.extenders = None
            self.gm = None
            
    @staticmethod
    def getGrandMotherElements(pointers):
        ## Identify Grandmother elements
        gmElements = []
        gmMatrix = []

        for i in range(len(pointers)):
            igmElements = []
            for j in range(i):
                ## Check to see if the Any Mother element is a Big Momma or "Bertha" Element
                if pointers[i].get_mother_element() is pointers[j].get_mother_element():
                    gm = pointers[i].get_mother_element()
                else:
                    gm = pointers[i].common_parent(pointers[j])
                # Append Match to Grand Mother Matrix
                igmElements.append(gm)

                # Check to see if this is a new grand mother element,
                # if so append to the gmElements list of unique grandmother elements 
                if gm not in gmElements:
                    gmElements.append(gm)

            # Append Matrix Row
            gmMatrix.append(igmElements)

        grandMotherMatrix = np.matrix(gmMatrix)
        return (gmElements, grandMotherMatrix)
    
    @staticmethod
    def distinct_gm(gm, pts):
        if len(pts) == 0:
            return True
        if gm is pts[0].get_mother_element():
            return False
        else:
            return ContactScraperVerifier.distinct_gm(gm, pts[1:])

# The Extender

In [6]:
class Extender(object):
    def __init__(self, gm, pointer):
        self.gm = gm
        self.vp = pointer
        
        self.tag_nathans()
        self.tag_toms()
        
        self.tom_missile = self.mother_to_tom_missile()
        self.tom_rocket = self.nathan_to_tom_rocket()
        self.nathan_shuttle = self.nathan_to_element()
        
        self.nathan_missile = self.mother_to_nathan_missile()
        self.nathan_rocket = self.tom_to_nathan_rocket()
        self.tom_shuttle = self.tom_to_element()
        
        #self.reset_tree()
        
    ## Tagging Functions ------------------------------------ 
    ##
    def tag_nathans(self):
        if self.vp.get_mother_element() is self.vp.nathan.parent:  ## Gotta Catch those first elements before the get away
            self.vp.nathan.parent['nathan'] = 0
        else:
            return Extender.parent_cycle_up(self.vp.get_mother_element(), self.vp.nathan.parent, 'nathan', 0)

    def tag_toms(self):
        if self.vp.get_mother_element() is self.vp.tom.parent:  ## Gotta Catch those first elements before the get away
            self.vp.tom.parent['tom'] = 0
        else:
            return Extender.parent_cycle_up(self.vp.get_mother_element(), self.vp.tom.parent, 'tom', 0)
    
    def reset_tree(self):
        ## Clear all tom, nathan and sib atributes
        for tomTag in self.gm.find_all(Extender.has_tom):
            del tomTag['tom']    
        for nathanTag in self.gm.find_all(Extender.has_nathan):
            del nathanTag['nathan'] 
        for sibTag in self.gm.find_all(Extender.has_sib):
            del sibTag['sib']
    
    @staticmethod
    def parent_cycle_up(motherElement, element, atr, num):
        ## We tag up THOUGH the MotherELement
        if element is motherElement:
            element[atr] = num
            #element['sib'] = atr
        else:
            element[atr] = num
            return Extender.parent_cycle_up(motherElement, element.parent, atr, num + 1)
    
    @staticmethod
    def contents_position(elm):
        return Extender.contents_position_loop(elm, 0)

    @staticmethod
    def contents_position_loop(elm, num):
        if len(list(elm.previous_siblings)) == 0:
            return num
        else:
            return Extender.contents_position_loop(elm.previous_sibling, num + 1)
    
    @staticmethod
    def has_tom(tag):
        try:
            return 'tom' in tag.attrs
        except AttributeError:
            return False

    @staticmethod
    def has_nathan(tag):
        try:
            return 'nathan' in tag.attrs
        except AttributeError:
            return False

    @staticmethod
    def has_sib(tag):
        try:
            return 'sib' in tag.attrs
        except AttributeError:
            return False
        
    @staticmethod
    def check_siblings(sibs, attFunc):
        for sib in sibs:
            if attFunc(sib):
                return True
        return False
    
    ## Shuttle Functions ------------------------------------
    ##
    def nathan_to_element(self):
        return lambda start: start.contents[Extender.contents_position(self.vp.nathan)]
    
    def tom_to_element(self):
        return lambda start: start.contents[Extender.contents_position(self.vp.tom)]
    
    ## Missle Functions ------------------------------------
    ##
    def mother_to_tom_missile(self):
        motherElement = self.vp.get_mother_element()

        if self.vp.tom is motherElement:
            return lambda start: start
        else:
            return lambda start: Extender.cycle_up_mother(self.vp.tom.parent, motherElement, start).contents[Extender.contents_position(self.vp.tom)]

    def mother_to_nathan_missile(self):
        motherElement = self.vp.get_mother_element()

        if self.vp.nathan is motherElement:
            return lambda start: start
        else:
            return lambda start: Extender.cycle_up_mother(self.vp.nathan.parent, motherElement, start).contents[Extender.contents_position(self.vp.nathan)]

    @staticmethod
    def cycle_up_mother(elm, motherElement, start):
        if elm is motherElement:
            return start
        else:
            return Extender.cycle_up_mother(elm.parent, motherElement, start).contents[Extender.contents_position(elm)]
        
    
    ## Rocket Functions ------------------------------------
    ##
    def nathan_to_tom_rocket(self):
        ## Compute the route from nathan startnode to tom by starting at tom and recursing up, accross and then down
        ## looing for nathan = 0 attribute
        
        return lambda start: Extender.cycle_up(self.vp.tom, 'nathan', start)

    def tom_to_nathan_rocket(self):
        ## Compute the route from tom startnode to nathan by starting at nathan and recursing up, accross and then down
        ## looing for tom = 0 attibute
        
        return lambda start: Extender.cycle_up(self.vp.nathan, 'tom', start)
    
    @staticmethod
    def cycle_up(elm, to, start):
        ## Set Attribut Search Function
        attrFunc = Extender.has_tom if to == 'tom' else Extender.has_nathan

        ## FIRST check yo'self
        if attrFunc(elm):  ## Switch Direction but don't move
            return Extender.cycle_down(elm, to, start) 
        ## SECOND look Left for shoulder in siblings
        elif Extender.check_siblings(elm.previous_siblings, attrFunc):
            return Extender.cycle_left(elm.previous_sibling, to, start).next_sibling
        ## Then look right
        elif Extender.check_siblings(elm.next_siblings, attrFunc):
            return Extender.cycle_right(elm.next_sibling, to, start).previous_sibling
        ## if nothing here go up
        else:
            return Extender.cycle_up(elm.parent, to, start).contents[Extender.contents_position(elm)]

    @staticmethod
    def cycle_left(elm, to, start):
        attrFunc = Extender.has_tom if to == 'tom' else Extender.has_nathan

        ## if this is the shoulder change functions but dont move
        if attrFunc(elm):
            return Extender.cycle_down(elm, to, start)
        ## if not check the element to the left
        else:
            return Extender.cycle_left(elm.previous_sibling, to, start).next_sibling

    @staticmethod
    def cycle_right(elm, to, start):
        attrFunc = Extender.has_tom if to == 'tom' else Extender.has_nathan

        ## if this is the shoulder change functions but dont move
        if attrFunc(elm):
            return Extender.cycle_down(elm, to, start)
        ## if not check the element to the left
        else:
            return Extender.cycle_right(elm.next_sibling, to, start).previous_sibling

    @staticmethod
    def cycle_down(elm, to, start):
        return Extender.cycle_down_loop(elm[to], start)

    @staticmethod
    def cycle_down_loop(dist, start):
        ## Start Node Condition
        if dist == 0:
            return start
        else: 
            return Extender.cycle_down_loop(dist - 1, start).parent
 
        

# Extractor Classes

In [7]:
class Extractor(Extender):
    nathanStartType = '__nathanStarts__'
    tomStartType = '__tomStarts__'
    motherStartType = '__motherStarts__'
    
    def __init__(self, gm, pointer):
        Extender.__init__(self, gm, pointer)
        self.startBlock = StartBlockNoMother(gm, pointer)
        self.startsType, self.starts = self.get_starts()
        self.resultSet = NewPointerSet()
        self.test_starts()
        self.reset_tree()
        
    def get_starts(self):
        return self.startBlock.get_optimal_starts()
    
    def test_starts(self):
        return [self.test_start(self.startsType, st) for st in self.starts]
        
    def test_start(self, startType, start):
        if startType == Extractor.nathanStartType:   ## Nathan Start Case, Nathan Shuttle, Tom Rocket to 
            np = NewPointer(start, self.nathan_shuttle, self.tom_rocket)
            self.resultSet.addPointer(np)
            return np
        if startType == Extractor.tomStartType:   ## Tom Start Case, Tom Shuttle, Nathan Rocket 
            np = NewPointer(start, self.nathan_rocket, self.tom_shuttle)
            self.resultSet.addPointer(np)
            return np
        if startType == Extractor.motherStartType:   ## Mother Start Case, Missile for Missle for Nathan, Missile for Tom 
            np = NewPointer(start, self.nathan_missile, self.tom_missile)
            self.resultSet.addPointer(np)
            return np
    
    def get_result_set(self):
        return self.resultSet
    
    def quick_report(self):
        print(str(self.resultSet))

In [8]:
class ExtractorNoMother(Extractor):
    nathanStartType = '__nathanStarts__'
    tomStartType = '__tomStarts__'
    motherStartType = '__motherStarts__'
    
    def __init__(self, gm, pointer):
        Extractor.__init__(self, gm, pointer)
        self.startBlock = StartBlockNoMother(gm, pointer)
        self.startsType, self.starts = self.get_starts()
        self.reset_tree()
        
    def get_starts(self):
        return self.startBlock.get_optimal_starts()

# StartBlock Classes

In [9]:
class StartBlock(object):
    def __init__(self, gm, pointer):
        self.vp = pointer
        self.gm = gm
        
        self.nathanNameStarts = gm.find_all(pointer.nathan.parent.name)
        self.noNathanNameStarts = len(self.nathanNameStarts)
        self.tomNameStarts = gm.find_all(pointer.tom.parent.name)
        self.noTomNameStarts = len(self.tomNameStarts)
        self.motherNameStarts = gm.find_all(pointer.get_mother_element().name)
        self.noMotherNameStarts = len(self.motherNameStarts)
        
        self.noNathanClasses = self.get_no_nathan_classes()
        self.noTomClasses = self.get_no_tom_classes()
        self.noMotherClasses = self.get_no_mother_classes()
        
    def get_no_nathan_classes(self):
        if 'class' in self.vp.nathan.parent.attrs:
            return len(self.vp.nathan.parent['class'])
        else:
            return 0
    
    def get_no_tom_classes(self):
        if 'class' in self.vp.tom.parent.attrs:
            return len(self.vp.tom.parent['class'])
        else:
            return 0        
    
    def get_no_mother_classes(self):
        if 'class' in self.vp.get_mother_element().attrs:
            return len(self.vp.get_mother_element()['class'])
        else:
            return 0
        
    def get_nathan_class_starts(self):
        return self.gm.find_all(class_=self.vp.nathan.parent['class'])
    
    def get_tom_class_starts(self):
        return self.gm.find_all(class_=self.vp.tom.parent['class'])
    
    def get_mother_class_starts(self):
        return self.gm.find_all(class_=self.vp.get_mother_element()['class'])
    
    def get_optimal_starts(self):
        maxNoClasses = max([self.noNathanClasses, self.noTomClasses, self.noMotherClasses])
        minNoNameStarts = min([self.noNathanNameStarts, self.noTomNameStarts, self.noMotherNameStarts])
        
        #dispatch
        if maxNoClasses > 0: ## There are classes to choose better starts
            if self.noMotherClasses == maxNoClasses:
                return '__motherStarts__', self.get_mother_class_starts()
            elif self.noNathanClasses == maxNoClasses:
                return '__nathanStarts__', self.get_nathan_class_starts()
            else:
                return '__tomStarts__', self.get_tom_class_starts()
        else:  ## There are no classes to test mus go by the minimun number of name matches
            if self.noMotherNameStarts == minNoNameStarts:
                return '__motherStarts__', self.motherNameStarts
            elif self.noNathanNameStarts == minNoNameStarts:
                return '__nathanStarts__', self.nathanNameStarts
            else:
                return '__tomStarts__', self.tomNameStarts

In [10]:
class StartBlockNoMother(StartBlock):
    def __init__(self, gm, pointer):
        StartBlock.__init__(self, gm, pointer)
    
    def get_optimal_starts(self):
        ## Unlike get_optimal_starts from the inheritted StartBlock this one does not return mother starts
        maxNoClasses = max([self.noNathanClasses, self.noTomClasses])
        minNoNameStarts = min([self.noNathanNameStarts, self.noTomNameStarts])
        
        #dispatch
        if maxNoClasses > 0: ## There are classes to choose better starts
            if self.noNathanClasses == maxNoClasses:
                return '__nathanStarts__', self.get_nathan_class_starts()
            else:
                return '__tomStarts__', self.get_tom_class_starts()
        else:  ## There are no classes to test mus go by the minimun number of name matches
            if self.noNathanNameStarts == minNoNameStarts:
                return '__nathanStarts__', self.nathanNameStarts
            else:
                return '__tomStarts__', self.tomNameStarts

# NewPointer Class

In [11]:
class NewPointer(object):
    def __init__(self, start, nathanTest, tomTest):
        self.nathanRoute = nathanTest
        self.tomRoute = tomTest
        self.start = start
        
        ## Execute Test
        self.nathan = self.test_nathan()
        self.tom = self.test_tom()
        
        self.output = ContactSheetOutput('New Pointer For: %s start' % str(start))
        
    def test_nathan(self):
        try:
            result = self.nathanRoute(self.start)
            if type(result) is not NavigableString:
                result = None
        except:
            result = None
            
        return result
    
    def test_tom(self):
        try:
            result = self.tomRoute(self.start)
            if type(result) is not NavigableString:
                result = None
        except:
            result = None
            
        return result
    
    def get_tom(self):
        return self.tom
    
    def get_first_last_name(self):
        passes = ['Mr', 'MR', 'Miss', 'Mrs', 'MRS', 'MS', 'Ms', 'Dr']
        nameWords = self.nathan.split(' ')
        spaceFiltered = [word for word in nameWords if word]
        ## loop through name words until you encounter the first two full words that are not passes

        firstNamePt = None
        lastNamePt = None

        for i in range(len(spaceFiltered)):
            if (spaceFiltered[i] not in passes) and ('.' not in spaceFiltered[i]):
                firstNamePt = i
                break

        for j in range(i+1,(len(spaceFiltered))):
            if (spaceFiltered[j] not in passes) and ('.' not in spaceFiltered[j]):
                lastNamePt = j
                break
        try:        
            firstName = ' '.join(spaceFiltered[:firstNamePt + 1])
            lastName = ' '.join(spaceFiltered[lastNamePt:])  
        except TypeError:
            return None, None

        if lastName: ## Last Name was Defined, return Tuple
            return firstName, lastName
        else:        ## Last Name was not Properly Defined result use first Name only
            return firstName, None
        
    def get_clean_tom(self):
        ## Returns a cleaned up version of the title
        tomWords = self.tom.split(' ')
        #filter out spaces
        return ' '.join([word for word in tomWords if word])
    
    def no_tom_words(self):
        return NewPointer.word_count(self.tom)
    
    def no_nathan_words(self):
        return NewPointer.word_count(self.nathan)
    
    def get_output_dict(self):
        outputDict = {}
        outputDict['firstName'], outputDict['lastName'] = self.get_first_last_name()
        outputDict['title'] = self.get_clean_tom()
        return outputDict
    
    @staticmethod
    def word_count(field):
        ## Returns a cleaned up version of the title
        fieldWords = field.split(' ')
        #filter out spaces
        return len([word for word in fieldWords if word])
    
    def get_nathan(self):
        return self.nathan

# NewPointerSet and MergeSetClasses

In [12]:
class NewPointerSet(object):
    def __init__(self):
        self.newPointers = []
        
    def addPointer(self, newPointer):
        self.newPointers.append(newPointer)
        
    def __str__(self):
        s = ''
        for pt in self.newPointers:
            s += ('%s     as     %s\n' % (str(pt.get_nathan()), str(pt.get_tom())))
        return s
    
    def get_pointers(self):
        return self.newPointers

In [13]:
class MergeSet(NewPointerSet):
    
    nathanWordLimit = 12
    tomWordLimit = 20
    
    def __init__(self, vps):
        NewPointerSet.__init__(self)
        self.verifiedPointers = vps
        self.output = NewContactSheetOutput()
        
    def merge_pointers(self, newPointerSet):
        for np in newPointerSet.get_pointers():
            self.add_pointer(np)
    
    def add_pointer(self, newPointer):
        ## Pointer has to meet certain conditions to be added to the MergeSet
        if MergeSet.has_a_failed_route(newPointer):
            return False
        if MergeSet.is_nothing(newPointer.tom) or MergeSet.is_nothing(newPointer.nathan):
            return False
        if MergeSet.is_an_email(newPointer.tom) or MergeSet.is_an_email(newPointer.nathan):
            return False
        if MergeSet.is_too_long(newPointer):
            return False
        if MergeSet.fails_name_test(newPointer):
            return False
        if self.a_match_to_verified_pointers(newPointer):
            return False
        if self.a_match_in_set(newPointer):
            return False
        
        ## Add to MergeSetCase
        self.newPointers.append(newPointer)
        
    def write_new_contacts(self):
        self.output.output_batch_row([np.get_output_dict() for np in self.get_pointers()])
        
    @staticmethod
    def fails_name_test(pointer):
        firstName, lastName = pointer.get_first_last_name()
        return not (firstName, lastName)
    
    @staticmethod
    def is_too_long(pointer):
        if pointer.no_nathan_words() > MergeSet.nathanWordLimit:
            return True
        if pointer.no_tom_words() > MergeSet.tomWordLimit:
            return True
        return False
                                                    
    @staticmethod
    def is_an_email(pointerWord):
        if '@' in pointerWord:
            return True
        return False
    
    @staticmethod                                                   
    def is_nothing(pointerWord):
        letters = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'       
        for char in pointerWord:
            if char in letters:
                return False
        return True 
    
    @staticmethod
    def has_a_failed_route(pointer):
        if not pointer.tom:
            return True
        if not pointer.nathan:
            return True
        return False
    
    def a_match_in_set(self, pointer):
        for np in self.newPointers:
            if pointer.nathan is np.nathan:
                return True
            if pointer.tom is np.tom:
                return True
        return False
    
    def a_match_to_verified_pointers(self, pointer):
        for vp in self.verifiedPointers:
            if pointer.nathan is vp.nathan:
                return True
            if pointer.tom is vp.tom:
                return True
        return False

# The Dorito and The Chip

In [14]:
class Dorito(object):
    def __init__(self, gm, pointers):
        self.grandMotherElement = gm
        self.verifiedPointers = pointers
        self.scrapePointers = Dorito.filterPointers(pointers)
        self.result = None
        self.finalPointers = MergeSet(pointers)
    
    def extract(self):
        ## This class will test the first One
        try:
            self.x = Extractor(self.grandMotherElement, self.scrapePointers[0])
            self.result = self.x.get_result_set()
        except IndexError:
            print("No Scrapable Matches")
        
    def merge_result(self):
        if self.result:
            self.finalPointers.merge_pointers(self.result)
            if not str(self.finalPointers):
                print("Nothing passed Merge")
        else:
            print("Nothing to Merge")    
        self.report()
        
    def merge_result_no_report(self):
        if self.result:
            self.finalPointers.merge_pointers(self.result)
            if not str(self.finalPointers):
                print("Nothing passed Merge")
        else:
            print("Nothing to Merge")    
        self.report()
            
    def report(self):
        print('Verified Pointers         \t\t\t\t\t\t%s' % len(self.verifiedPointers))
        print('Scrape Pointers           \t\t\t\t\t\t%s' % len(self.scrapePointers))
        print('Start Pointers            \t\t\t\t\t\t%s' % (len(self.result.get_pointers()) if self.result else 0))
        try:
            print('Start Type                \t\t\t\t%s' % self.x.startsType)
            print('Merged (Filtered) Pointers\t\t\t\t\t\t%s' % len(self.finalPointers.get_pointers()))
            print(self.finalPointers)
            
        except AttributeError:
            print('Start Type                \t\t\t\tNO EXTRACTORS')
    
    def write_contacts(self):
        self.finalPointers.write_new_contacts()
    
    @staticmethod
    def filterPointers(vps):
        return [pointer for pointer in vps if pointer.mary_here() and pointer.nathan_here]

In [15]:
class Chip(Dorito):
    def __init__(self, gm, pointers):
        Dorito.__init__(self, gm, pointers)
        Dorito.extract(self)
        Dorito.merge_result(self)
        Dorito.write_contacts(self)
            

# NewContactSheetOutput

In [16]:
class NewContactSheetOutput(ContactSheetOutput):
    dummyRecord = None
    
    def __init__(self):
        ContactSheetOutput.__init__(self, NewContactSheetOutput.dummyRecord['Account Name'].to_string(index=False))
        
    def output_batch_row(self, newContactInfo):
        """Google Sheets API Code.  Since we are in NewContacts Sheet Output, the batch out put function is tasked
           with weaving the information from the new contact with the organization information from the other columns.
           These will be filled in to accompany First Name Last Name and Tittle for the new contacts 
        """
        credentials = ContactSheetOutput.get_credentials()
        http = credentials.authorize(smgs.httplib2.Http())
        discoveryUrl = ('https://sheets.googleapis.com/$discovery/rest?'
                        'version=v4')
        service = smgs.discovery.build('sheets', 'v4', http=http,
                                  discoveryServiceUrl=discoveryUrl)

        spreadsheet_id = '1p1LNyQhNhDBNEOkYQPV9xcNRe60WDlmnuiPp78hxkIs'
        value_input_option = 'RAW'
        rangeName = 'Samples!A' + str(ContactSheetOutput.currentRow)
        values = [NewContactSheetOutput.weaveContactInfo(nci) for nci in newContactInfo]
        body = {
              'values': values
        }

        try:
            result = service.spreadsheets().values().update(spreadsheetId=spreadsheet_id, range=rangeName,
                                                        valueInputOption=value_input_option, body=body).execute()
        except:
            print('Missed Row Output')
        else:
            ContactSheetOutput.currentRow += len(values)

        return result
    
    @staticmethod
    def weaveContactInfo(nci):
        return [NewContactSheetOutput.dummyRecord['Account ID'].to_string(index=False),
                NewContactSheetOutput.dummyRecord['Account Name'].to_string(index=False),
                '',
                nci['firstName'],
                nci['lastName'],
                nci['title'],
                '',
                NewContactSheetOutput.dummyRecord['Mailing Street'].to_string(index=False),
                NewContactSheetOutput.dummyRecord['Mailing City'].to_string(index=False),
                NewContactSheetOutput.dummyRecord['Mailing State/Province'].to_string(index=False),
                NewContactSheetOutput.dummyRecord['Mailing Zip/Postal Code'].to_string(index=False),
                NewContactSheetOutput.dummyRecord['Mailing Country'].to_string(index=False),
                '',
                '',
                '',
                '',
                '',
                '',
                '',
                '',
                'New']
    
    @classmethod
    def set_Org_Fill_In_Fields(cls, record):
        NewContactSheetOutput.dummyRecord = record

In [17]:
class SurrogateErrorOutput(ContactSheetOutput):
    def __init__(self, name):
        ContactSheetOutput.__init__(self, name)
        
    def output_batch_row(self, rows, errorMessage):
        """Google Sheets API Code. Here at SurrogateErrorOutput we append the error message information to the end of each row 
        """
        credentials = ContactSheetOutput.get_credentials()
        http = credentials.authorize(smgs.httplib2.Http())
        discoveryUrl = ('https://sheets.googleapis.com/$discovery/rest?'
                        'version=v4')
        service = smgs.discovery.build('sheets', 'v4', http=http,
                                  discoveryServiceUrl=discoveryUrl)

        spreadsheet_id = '1p1LNyQhNhDBNEOkYQPV9xcNRe60WDlmnuiPp78hxkIs'
        value_input_option = 'RAW'
        rangeName = 'Samples!A' + str(ContactSheetOutput.currentRow)
        #values = rows
        values = [SurrogateErrorOutput.appendError(row, errorMessage) for row in rows]
        body = {
              'values': values
        }
        #print(values)
        try:
            result = service.spreadsheets().values().update(spreadsheetId=spreadsheet_id, range=rangeName,
                                                        valueInputOption=value_input_option, body=body).execute()
        except:
            print('Missed Row Output')
        else:
            ContactSheetOutput.currentRow += len(values)

        return result
        
    @staticmethod
    def appendError(row, em):
        row.extend(['', '', '', '', '', '', 'Not Verified', em])
        return row

In [27]:
class ContactCollector(ContactScraperVerifier):
    def __init__(self, org):
        
        print('\nScraping %s' % org)
        
        try:
            ContactScraperVerifier.__init__(self, org)
            self.write_contact_pointers()
        except:
            ## Link DID NOT Open
            print("Link DID NOT Open Properly")
            self.errorOutput = SurrogateErrorOutput(org)
            
            x = self.errorOutput.output_batch_row([[self.records.iloc[[ind]][key].to_string(index=False) for key in self.errorOutput.get_contact_keys()] for ind in range(len(self.records))], 'Link did not open')

        else:
            try:
                if self.noGm == 1:  ## Single Grandmother Case
                    self.c = Chip(self.gm, self.vPointers)
                elif (self.noGm == 0) and (len(self.vPointers) == 1): ## No GrandMother Single Verfied Pointer - Try this!
                    self.c = Chip(self.vPointers[0].get_mother_element().parent, self.vPointers)
                else:
                    print('NO Extenders')
            except:
                ## Some Mysterious Extender/ScraperError no Need for serrogate output
                print('Extender/Scraper Error - NO Extenders')

In [19]:
orgRecords[2]['Organization']

'American Academy of Pediatrics (AAP)'

In [20]:
orgsForToday = ['National Association for Multi-Ethnicity In Communications (NAMIC)',
                'Association for Women in Science',
                'Brain Injury Association of America',
                'American Society of Home Inspectors',
                'NAADAC, the Association for Addiction Professionals',
                'American Public Transportation Association',
                'Indiana Soybean Alliance',
                'Associated Builders and Contractors (ABC)',
                'National Association of Social Workers',
                'American Marketing Association (AMA)']

errorCases = ['Arizona School Boards Association',
              'American Association of Diabetes Educators']

In [28]:
org  = orgsForToday[7]

vh = ContactCollector('Association for Accounting Marketing')


Scraping Association for Accounting Marketing
['Wed Oct 25, 2017  10:27:32', '1813', 'ms', 'Good']
Verified Pointers         						2
Scrape Pointers           						2
Start Pointers            						3
Start Type                				__nathanStarts__
Merged (Filtered) Pointers						1
Jessica Kiser     as     Association Coordinator



In [29]:
for org in orgsForToday:
    vh = ContactCollector(org)


Scraping National Association for Multi-Ethnicity In Communications (NAMIC)
['Wed Oct 25, 2017  10:27:58', '7.864', 's', 'Good']
Verified Pointers         						3
Scrape Pointers           						3
Start Pointers            						8
Start Type                				__nathanStarts__
Merged (Filtered) Pointers						5
Eglon E. Simons     as     President and CEO
James C. Jones     as     Senior Vice President, Education and Diversity Solutions
Sandra Girado     as     Vice President, Events and Partner Relations
Brandon D. Hewlett     as     Project Coordinator
Davana O'Brien     as     Administrative Assistant


Scraping Association for Women in Science
['Wed Oct 25, 2017  10:28:04', '2.801', 's', 'Good']
No Fred
NO Extenders

Scraping Brain Injury Association of America
['Wed Oct 25, 2017  10:28:08', '989', 'ms', 'Good']
Verified Pointers         						4
Scrape Pointers           						4
Start Pointers            						10
Start Type                				__tomStarts__
Merged (Filtered) Pointer

No Fred
No Fred
NO Extenders

Scraping National Association of Social Workers
['Wed Oct 25, 2017  10:28:39', '1032', 'ms', 'Good']
No Fred
No Fred
Verified Pointers         						2
Scrape Pointers           						2
Start Pointers            						3
Start Type                				__nathanStarts__
Merged (Filtered) Pointers						1
Heather Bradley-Geary, MSW     as      Secretary 


Scraping American Marketing Association (AMA)
['Wed Oct 25, 2017  10:28:45', '1764', 'ms', 'Good']
No Fred
No Reggies
No Fred
No Fred
No Fred
No Fred
Verified Pointers         						5
Scrape Pointers           						4
Start Pointers            						12
Start Type                				__nathanStarts__
Merged (Filtered) Pointers						2
Russ Klein​​     as     Chief Executive Officer - 
Michael Pemberton​     as     Accounting - 



In [ ]:
for orgRec in orgRecords:
    vh = ContactCollector(orgRec['Organization'])

['Wed Oct 25, 2017  03:54:48', '14.234', 's', 'Good']
No Larry
No Fred
No Larry
No Larry
No Larry
No Larry
No Fred
No Fred
No Fred
No Larry
Nothing passed Merge
Verified Pointers         						1
Scrape Pointers           						1
Start Pointers            						1
Start Type                				__nathanStarts__
Merged (Filtered) Pointers						0

['Wed Oct 25, 2017  03:54:57', '3.127', 's', 'Good']
No Fred
No Fred
No Fred
No Fred
No Fred
No Larry
No Larry
No Larry
No Fred
No Fred
No Fred
No Larry
No Fred
No Larry
Nothing passed Merge
Verified Pointers         						5
Scrape Pointers           						5
Start Pointers            						49
Start Type                				__nathanStarts__
Merged (Filtered) Pointers						0

['Wed Oct 25, 2017  03:55:04', '3.136', 's', 'Good']
No Fred
No Fred
No Larry
No Fred
No Fred
No Larry
No Larry
No Fred
No Larry
No Fred
No Fred
No Larry
No Fred
No Fred
No Fred
NO Extenders
['Wed Oct 25, 2017  03:55:09', '2.167', 's', 'Good']
No Fred
No Fred
No Fred
No Fred
No 

['Wed Oct 25, 2017  04:00:02', '3.255', 's', 'Good']
No Fred
No Fred
No Fred
No Fred
NO Extenders
['Wed Oct 25, 2017  04:00:09', '4.226', 's', 'Good']
No Fred
No Fred
No Fred
No Fred
No Fred
No Fred
No Fred
No Fred
No Fred
No Fred
No Fred
No Fred
No Fred
No Fred
NO Extenders
['Wed Oct 25, 2017  04:00:13', '1695', 'ms', 'Good']
Nothing passed Merge
Verified Pointers         						4
Scrape Pointers           						2
Start Pointers            						9
Start Type                				__tomStarts__
Merged (Filtered) Pointers						0

['Wed Oct 25, 2017  04:00:20', '3.902', 's', 'Good']
No Fred
Nothing passed Merge
Verified Pointers         						1
Scrape Pointers           						1
Start Pointers            						1
Start Type                				__nathanStarts__
Merged (Filtered) Pointers						0

['Wed Oct 25, 2017  04:00:24', '1270', 'ms', 'Good']
No Fred
No Larry
No Fred
No Fred
No Fred
No Fred
Verified Pointers         						1
Scrape Pointers           						1
Start Pointers            						7

Verified Pointers         						3
Scrape Pointers           						3
Start Pointers            						26
Start Type                				__nathanStarts__
Merged (Filtered) Pointers						20
Brig. Gen. Joseph Schroedel,     as     Executive Director
Allison Ingram     as      Director of Operations, Finance andAdministration
Kathy Off, CGMP, PMP     as      Strategic and Stakeholder Relations
Martina Salazar     as      Administrative Assistant
Allison Ingram                                                                       as      Director of Operations, Finance andAdministration
Will Hoffman     as      Senior Accountant
 Ekaterina (Katia) Khoreva     as     Staff Accountant
Otis Carter     as      Logistics and Administrative Specialist
Ann McLeod, CEM, CAE     as     Meetings and Marketing Director
Amira Kruyne     as     Meetings & Exhibits Manager
Marlayna Bush     as      Sales and Business Development Coordinator
Jacqueline Barrett     as     Registration & Customer Support Manag

In [32]:
org  = errorCases[0]

vh = ContactCollector(org)

Link DID NOT Open Properly
[['001U000000KzQmv', 'Arizona School Boards Association', '003U000000Mh58Z', 'Chuck', 'Essigs', 'Interim Executive Director', 'cessigs@azsba.org', '', '', 'AZ', '', '', '(602) 254-1100', '', '', '', '', '', '', '', 'Not Verified', 'Link did not open']]


In [50]:
print(vh.c.result)

AttributeError: 'ContactCollector' object has no attribute 'c'

In [129]:
vh.write_contact_pointers()

In [130]:
c = Chip(vh.gm, vh.vPointers)

Verified Pointers         						5
Scrape Pointers           						4
Start Pointers            						12
Start Type                				__nathanStarts__
Merged (Filtered) Pointers						2
Russ Klein​​     as     Chief Executive Officer - 
Michael Pemberton​     as     Accounting - 



In [121]:
cc = Chip(vh.vPointers[0].get_mother_element().parent, vh.vPointers)

Verified Pointers         						1
Scrape Pointers           						1
Start Pointers            						19
Start Type                				__nathanStarts__
Merged (Filtered) Pointers						15
Chantel Rammel     as     Sr. Director of Corporate Services -   
Ed Ebert     as     Sr. Director of Grain Production and Utilization -   
Andy Tauer     as     Director of Livestock -   
Alyson Wells     as     Director of  Production and Environment -   
Ken Parrent     as     Director of Biofuels -   
Hannah Vorsilak     as     Director of Education and Training -   
Melanie Batalis     as     Marketing Program Manager -   
Cory Harris     as     Public Affairs Program Manager -   
Lauren Taylor     as     Marketing Program Manager -   
April Shepherd     as     Membership Program Manager -   
Robyn Shelley     as     Program Manager & Research Coordinator -   
Angela Proctor     as     Accounting Assistant -   
Kris Sturtz     as     Office Manager -   
Carol Collins     as     Receptionist and Offi

In [63]:
NewContactSheetOutput.dummyRecord

,Account ID,Account Name,Contact ID,Contact Source,Email,First Name,Last Name,Mailing City,Mailing Country,Mailing State/Province,Mailing Street,Mailing Zip/Postal Code,Notes,Phone,Status,Title
1263,001U000000DBVD4,National Association of Social Workers,003U0000019av9p,,president@naswdc.org,Darrell P.,Wheeler,Washington,,DC,750 First St. NE Suite 700,20002-4241,,(202) 408-8600,,President


In [64]:
NSO = NewContactSheetOutput()

In [65]:
NSO.name

'National Association of Social Workers'

In [32]:
[[vh.records.iloc[[ind]][key].to_string(index=False) for key in vh.output.get_contact_keys()] for ind in range(len(vh.records))]

[['001U000000KzQmv',
  'Arizona School Boards Association',
  '003U000000Mh58Z',
  'Chuck',
  'Essigs',
  'Interim Executive Director',
  'cessigs@azsba.org',
  '',
  '',
  'AZ',
  '',
  '',
  '(602) 254-1100',
  '']]

In [33]:
[vh.records.iloc[[ind]] for ind in range(len(vh.records))]

[           Account ID                       Account Name       Contact ID  \
 1791  001U000000KzQmv  Arizona School Boards Association  003U000000Mh58Z   
 
      Contact Source              Email First Name Last Name Mailing City  \
 1791                 cessigs@azsba.org      Chuck    Essigs                
 
      Mailing Country Mailing State/Province Mailing Street  \
 1791                                     AZ                  
 
      Mailing Zip/Postal Code Notes           Phone Status  \
 1791                                (602) 254-1100          
 
                            Title  
 1791  Interim Executive Director  ]

In [22]:
d = Dorito(vh.gm, vh.pointers)

In [23]:
d.extract()

In [24]:
d.merge_result()

Verified Pointers         						4
Scrape Pointers           						4
Start Pointers            						10
Start Type                				__tomStarts__
Merged (Filtered) Pointers						5
William M. Dane     as                  Public Affairs Manager
Tiffany Epley     as                  ACBIS Assistant
Sarah Gaffney     as                  BIAA - Maine Program Coordinator
Carrie Mosher     as                  Sales Manager
Steve Wade     as                  BIAA-Maine Executive Director



In [46]:
vh.records

,Account ID,Account Name,Contact ID,Contact Source,Email,First Name,Last Name,Mailing City,Mailing Country,Mailing State/Province,Mailing Street,Mailing Zip/Postal Code,Notes,Phone,Status,Title
1263,001U000000DBVD4,National Association of Social Workers,003U0000019av9p,,president@naswdc.org,Darrell P.,Wheeler,Washington,,DC,750 First St. NE Suite 700,20002-4241,,(202) 408-8600,,President
1264,001U000000DBVD4,National Association of Social Workers,003U000001nZIej,,pjohnson@naswct.net,Patrick,Johnson,Washington,,DC,750 First St. NE Suite 700,20002-4241,,(202) 408-8600,,Treasurer
1265,001U000000DBVD4,National Association of Social Workers,003U000001nZIhT,,mjoyner@nasw-pa.org,Mildred,Joyner,Washington,,DC,750 First St. NE Suite 700,20002-4241,,(202) 408-8600,,Vice President
1266,001U000000DBVD4,National Association of Social Workers,003U00000161Ke9,,naswceo@naswdc.org,Angelo,McClain,,,DC,750 First St. NE,20002-4241,,(202) 408-8600,,Chief Executive Officer


In [50]:
vh.records.iloc[[0]]['Account Name'].to_string(index=False)

'National Association of Social Workers'